In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv3D
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.attention.multi_head_attention import regularization
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv3D, AveragePooling2D
from keras.layers import Dense, Flatten, Dropout, BatchNormalization

In [ ]:
vgg16 = Sequential()

vgg16.add(Conv2D(filters=64, kernel_size=3, padding='same', strides = (1,1), activation='relu', input_shape =(224,224,3)))
vgg16.add(Conv2D(filters=64, kernel_size=3, padding='same', strides = (1,1), activation='relu', input_shape =(224,224,3)))
vgg16.add(MaxPooling2D(pool_size=2))
vgg16.add(Conv2D(filters=128, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(Conv2D(filters=128, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2))
vgg16.add(Conv2D(filters=256, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(Conv2D(filters=256, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(Conv2D(filters=256, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2))
vgg16.add(Conv2D(filters=512, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(Conv2D(filters=512, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(Conv2D(filters=512, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2))
vgg16.add(Conv2D(filters=512, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(Conv2D(filters=512, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(Conv2D(filters=512, kernel_size=3, padding='same', strides = (1,1), activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2))


vgg16.add(Flatten())


vgg16.add(Dense(4096, activation="relu"))
vgg16.add(Dense(4096, activation="relu"))
vgg16.add(Dense(1000, activation="softmax"))

In [ ]:
vgg16.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_16 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_17 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_18 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

-----------

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Concatenate
from keras.layers.merging.concatenate import concatenate

In [ ]:
def inception(             x, filter_1x1, 
              filter_3x3_1x1, filter_3x3,
              filter_5x5_1x1, filter_5x5, 
              filter_maxpool) :

  Conv_1x1     = Conv2D(filter_1x1, kernel_size=1, padding='same', activation='relu')(x)
  
  Conv_3x3_1x1 = Conv2D(filter_3x3_1x1, kernel_size=1, padding='same', activation='relu')(x)
  Conv_3x3     = Conv2D(filter_3x3, kernel_size=3, padding='same', activation='relu')(Conv_3x3_1x1)

  Conv_5x5_1x1 = Conv2D(filter_5x5_1x1, kernel_size=1, padding='same', activation='relu')(x)
  Conv_5x5     = Conv2D(filter_5x5, kernel_size=5, padding='same', activation='relu')(Conv_5x5_1x1)

  max_pool     = MaxPooling2D(pool_size=3, strides=1, padding='same')(x)
  max_1x1      = Conv2D(filter_maxpool, kernel_size=1, padding='same', activation='relu')(max_pool)

  output       = Concatenate(axis=3)([Conv_1x1,Conv_3x3, Conv_5x5, max_1x1])
  return output

In [ ]:
input_layer = Input(shape=(224,224,3))

In [ ]:
inception(input_layer, 64, 96, 128, 16, 32, 32)

<KerasTensor: shape=(None, 224, 224, 256) dtype=float32 (created by layer 'concatenate_1')>

---------

In [ ]:
from keras.layers import Conv2D, BatchNormalization, Activation, Add, MaxPooling2D
from keras import Input

In [ ]:
def bottleneck_residual_block (X, filters, strides, reduce=False) :
  f1,f2,f3 = filters

  X_shortcut = X

  # 잔차블록 첫번째 층
  # shortcut on;
  if reduce == True :
    # 잔차블록 층에서 나중에 합해줄 shortcut 부분 할당
    X_shortcut = Conv2D(filters= f3, kernel_size = 1, strides= strides)(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)
    
    # 
    X = Conv2D(filters = f1, kernel_size =1, strides=strides, padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
  
  # shortcut off;
  else :
    X = Conv2D(filters = f1, kernel_size =1, strides=1, padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

  # 두번째 층
  X = Conv2D(filters = f2, kernel_size =3, strides=1, padding='same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  # 세번째 층
  X = Conv2D(filters = f3, kernel_size =1, strides=1, padding='same')(X)
  X = BatchNormalization(axis=3)(X)
  

  # 마지막 잔차 블록 합
  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X




In [ ]:
# X.shape

TensorShape([None, 55, 55, 64])

In [ ]:
input_layer = Input(shape=(224,224,3))

In [ ]:
input = (224, 224,3)

In [ ]:
print(input)

(224, 224, 3)


In [ ]:
a=Input(input)

In [ ]:

print(a)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_12'), name='input_12', description="created by layer 'input_12'")


In [ ]:
def ResNet50(input_layer) :


  # 1단계

  X = Conv2D(64, kernel_size=7, strides=2, padding='same')(input_layer)
  # print(X.shape)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D(3, strides=2, padding='same')(X)
  # print(X.shape)


  # 2단계 

  X = bottleneck_residual_block(X,(64,64,256), strides=1, reduce=True)
  X = bottleneck_residual_block(X,(64,64,256), strides=1)
  X = bottleneck_residual_block(X,(64,64,256), strides=1)


  # 3단계 

  X = bottleneck_residual_block(X,(128,128,512), strides=2, reduce=True)
  X = bottleneck_residual_block(X,(128,128,512), strides=1)
  X = bottleneck_residual_block(X,(128,128,512), strides=1)
  X = bottleneck_residual_block(X,(128,128,512), strides=1)


  # 4단계 

  X = bottleneck_residual_block(X,(256,256,1024), strides=2, reduce=True)
  X = bottleneck_residual_block(X,(256,256,1024), strides=1)
  X = bottleneck_residual_block(X,(256,256,1024), strides=1)
  X = bottleneck_residual_block(X,(256,256,1024), strides=1)
  X = bottleneck_residual_block(X,(256,256,1024), strides=1)
  X = bottleneck_residual_block(X,(256,256,1024), strides=1)



  # 5단계 

  X = bottleneck_residual_block(X,(512,512,2048), strides=2, reduce=True)
  X = bottleneck_residual_block(X,(512,512,2048), strides=1)
  X = bottleneck_residual_block(X,(512,512,2048), strides=1)

  # 평균 풀링

  X = AveragePooling2D((1,1))(X)

  # 출력층

  X = Flatten()(X)
  X = Dense(1000,activation = 'softmax')(X)

  return X



In [ ]:
ResNet50(input_layer)

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'dense')>

In [ ]:
# X = bottleneck_residual_block(input_layer,(64,64,256), strides=1, reduce=True)
# X = bottleneck_residual_block(X,[64,64,256], strides=2)
# X = bottleneck_residual_block(X,[64,64,256], strides=2)